In [4]:
import torch.nn as nn
import torch.nn.functional as F
from layers import *
from utils import *
import torch
from torch.autograd import Variable

from models import *
from tcn.tcn import TemporalConvNet

from sklearn import metrics
from sklearn.utils import shuffle, resample
from sklearn.model_selection import train_test_split, KFold

In [10]:
class dti_pred_net(nn.Module):
    def __init__(self, gcn_params, tcn_params, n_den1, n_den2, n_den3):

        super(dti_pred_net, self).__init__()

#         self.gcn = Concate_GCN(n_bfeat=gcn_params.n_bfeat, n_afeat=25,
#                             n_sgc1_1=gcn_params.n_sgc1_1, n_sgc1_2=gcn_params.n_sgc1_2, n_sgc1_3=gcn_params.n_sgc1_3, 
#                             n_sgc1_4=gcn_params.n_sgc1_4, n_sgc1_5=gcn_params.n_sgc1_5,
#                             n_sgc2_1=gcn_params.n_sgc2_1, n_sgc2_2=gcn_params.n_sgc2_2, n_sgc2_3=gcn_params.n_sgc2_3, 
#                             n_sgc2_4=gcn_params.n_sgc2_4, n_sgc2_5=gcn_params.n_sgc2_5,
#                             n_den1=gcn_params.n_den1, n_den2=gcn_params.n_den2,
#                             nclass=gcn_params.nclass, dropout=gcn_params.dropout, use_att = True, molfp_mode = 'sum')
        self.tcn = TemporalConvNet(tcn_params.input_size, tcn_params.num_channels, kernel_size=tcn_params.kernel_size, dropout=tcn_params.dropout)
#         self.den1 = nn.Linear(, )
#         self.den2 = nn.Linear(, )
#         self.den3 = nn.Linear(, 1)

    def forward(self, adjs, afms, bfts, OrderAtt, AromAtt, ConjAtt, RingAtt, xp):

#         fp = self.gcn(adjs, afms, bfts, OrderAtt, AromAtt, ConjAtt, RingAtt)
        pd = self.tcn(xp)
        print pd

#         x1 = torch.cat((fp, pd), dim=1)
#         x1 = F.relu(self.den1(x1))
#         x1 = F.relu(self.den2(x1))
#         y = F.sigmoid(self.den3(x1))

#         return y

class tcn_params():
    def __init__(self, input_size, num_channels, kernel_size, dropout):
        self.input_size = input_size
        self.num_channels = num_channels
        self.kernel_size = kernel_size
        self.dropout = dropout
        

In [ ]:
dataset = 'bindingdb'   # 'tox21', 'hiv'
EAGCN_structure = 'concate' #  'concate', 'weighted_ave'
write_file = True
n_den1, n_den2= 64, 32
if dataset == 'bindingdb':
    n_sgc1_1, n_sgc1_2, n_sgc1_3, n_sgc1_4, n_sgc1_5 = 30, 10, 10, 10, 10
    n_sgc2_1, n_sgc2_2, n_sgc2_3, n_sgc2_4, n_sgc2_5 = 60, 20, 20, 20, 20
    batch_size = 64
    weight_decay = 0.00001  # L-2 Norm
    dropout = 0.3
    random_state = 1
    num_epochs = 50
    learning_rate = 0.0005
if dataset == 'bindingdb':
    all_tasks = ['label']
tasks = all_tasks
tcn_params = tcn_params()

In [6]:
x_all,xp_all,y_all, target, sizes = load_data(dataset)
max_size = max(sizes)
x_all,xp_all, y_all = data_filter(x_all,xp_all, y_all, target, sizes, tasks)
x_all, xp_all,y_all = shuffle(x_all,xp_all,y_all, random_state=random_state)

X, x_test,Xp,xp_test, y, y_test = train_test_split(x_all,xp_all,y_all, test_size=0.1, random_state=random_state)
# print(x_all)
# print xp_all.size
del x_all,xp_all,y_all
test_loader = construct_loader(x_test,xp_test,y_test, target, batch_size)
del x_test,xp_test, y_test

n_bfeat = X[0][2].shape[0]


Loading TCNdata.csv dataset...
filted_atomtype_list_order: [6, 7, 8, 9, 16, 17, 35, 15, 53, 11, 5, 34, 14, 23, 'Others'], 
 filted_bondtype_list_order: ['6_6', '6_7', '6_8', '6_9', '6_16', '7_7', '6_17', '8_16', '7_16', '7_8', '6_35', '8_15', '6_15', '6_53', '16_16', '5_8', '6_14', '5_6', '9_16', '7_15', '8_23', '8_11', '6_34', 'Others']
Transfer mol to matrices
[<neural_fp.Node instance at 0x7ff92ae78248>, <neural_fp.Node instance at 0x7ff92ae78170>, <neural_fp.Node instance at 0x7ff92ae78518>, <neural_fp.Node instance at 0x7ff92ae76d40>, <neural_fp.Node instance at 0x7ff92ae769e0>, <neural_fp.Node instance at 0x7ff92ae76cf8>, <neural_fp.Node instance at 0x7ff92ae76f38>, <neural_fp.Node instance at 0x7ff92ae76680>, <neural_fp.Node instance at 0x7ff92ae76c68>, <neural_fp.Node instance at 0x7ff92ae76b48>, <neural_fp.Node instance at 0x7ff92ae77fc8>, <neural_fp.Node instance at 0x7ff92ae77ef0>, <neural_fp.Node instance at 0x7ff92ae77cf8>, <neural_fp.Node instance at 0x7ff92ae77b90>, <neu

[<neural_fp.Node instance at 0x7ff92ae7a8c0>, <neural_fp.Node instance at 0x7ff92ae7a3f8>, <neural_fp.Node instance at 0x7ff92ae7a1b8>, <neural_fp.Node instance at 0x7ff92ae7a4d0>, <neural_fp.Node instance at 0x7ff92ae7a368>, <neural_fp.Node instance at 0x7ff92b7c7f38>, <neural_fp.Node instance at 0x7ff9296375a8>, <neural_fp.Node instance at 0x7ff929637128>, <neural_fp.Node instance at 0x7ff929637518>, <neural_fp.Node instance at 0x7ff929637368>, <neural_fp.Node instance at 0x7ff929637758>, <neural_fp.Node instance at 0x7ff929637b48>, <neural_fp.Node instance at 0x7ff9296375f0>, <neural_fp.Node instance at 0x7ff929637710>, <neural_fp.Node instance at 0x7ff9296377a0>, <neural_fp.Node instance at 0x7ff929637a70>, <neural_fp.Node instance at 0x7ff929637c20>, <neural_fp.Node instance at 0x7ff929637e18>, <neural_fp.Node instance at 0x7ff929637f80>, <neural_fp.Node instance at 0x7ff929637fc8>, <neural_fp.Node instance at 0x7ff929637ef0>, <neural_fp.Node instance at 0x7ff929637830>, <neural_f

In [7]:
x_train, x_val,xp_train,xp_val, y_train, y_val = train_test_split(X,Xp, y, test_size=0.1, random_state=random_state)
train_loader = construct_loader(x_train,xp_train, y_train, target, batch_size)
validation_loader = construct_loader(x_val, xp_val,y_val, target, batch_size)
len_train = len(x_train)
del x_train,xp_train, y_train, x_val, xp_val,y_val

In [8]:
train_loader

In [9]:
dti = dti_pred_net()
for i, (adj, afm, btf, orderAtt, aromAtt, conjAtt, ringAtt,xp, labels) in enumerate(train_loader):
    print xp
    xp = Variable(xp)
    dti(adj, afm, btf, orderAtt, aromAtt, conjAtt, ringAtt,xp)
    


   77    71    83  ...     59    59    59
   77    87    83  ...     59    59    59
   77    82    76  ...     59    59    59
       ...          ⋱          ...       
   77    71    83  ...     59    59    59
   77    71    83  ...     59    59    59
   77    82    76  ...     59    59    59
[torch.DoubleTensor of size 64x600]


   77    71    83  ...     59    59    59
   77    82    80  ...     59    59    59
   77    71    83  ...     59    59    59
       ...          ⋱          ...       
   77    71    83  ...     59    59    59
   77    71    83  ...     59    59    59
   77    80    75  ...     59    59    59
[torch.DoubleTensor of size 17x600]

